In [2]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import hashlib
from PIL import Image
import numpy as np

# Encrypt the message using AES
def encrypt_message(message, password):
    key = hashlib.sha256(password.encode()).digest()
    iv = get_random_bytes(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(message.encode(), AES.block_size))
    return iv + ciphertext

# Embed encrypted data into image using LSB
def embed_data(image_path, data, output_path):
    image = Image.open(image_path).convert("RGB")
    data_bits = ''.join(format(byte, '08b') for byte in data) + '1111111111111110'
    np_pixels = np.array(image).flatten()

    if len(data_bits) > len(np_pixels):
        raise ValueError("Message too large to embed in image.")

    for i in range(len(data_bits)):
        np_pixels[i] = (np_pixels[i] & ~1) | int(data_bits[i])

    new_image = Image.fromarray(np_pixels.reshape(image.size[1], image.size[0], 3).astype('uint8'), "RGB")
    new_image.save(output_path)
    print("✅ Message successfully embedded in", output_path)

# Extract encrypted data from image
def extract_encrypted_data(stego_path):
    image = Image.open(stego_path).convert("RGB")
    flat = np.array(image).flatten()
    bits = ''
    for pixel in flat:
        bits += str(pixel & 1)
        if bits[-16:] == '1111111111111110':
            break
    byte_chunks = [bits[i:i+8] for i in range(0, len(bits)-16, 8)]
    return bytes([int(b, 2) for b in byte_chunks])

# === User Interaction ===
input_message = input("Enter your secret message: ")
original_key = input("Enter a password for encryption: ")
cover_image = "download.jpg"
output_image = "stego_image.png"

# Encrypt and embed
encrypted = encrypt_message(input_message, original_key)
embed_data(cover_image, encrypted, output_image)

# Re-enter key to view original message
print("\n🔁 Re-enter your password to unlock the message.")
reentered_key = input("Re-enter password: ")

if hashlib.sha256(original_key.encode()).digest() == hashlib.sha256(reentered_key.encode()).digest():
    extracted_bytes = extract_encrypted_data(output_image)
    iv = extracted_bytes[:16]
    ciphertext = extracted_bytes[16:]
    key = hashlib.sha256(reentered_key.encode()).digest()
    cipher = AES.new(key, AES.MODE_CBC, iv)
    try:
        decrypted = unpad(cipher.decrypt(ciphertext), AES.block_size).decode()
        print("\n🔓 Original message:", decrypted)
    except:
        print("❌ Decryption failed. Encrypted data may be corrupted.")
else:
    print("❌ Incorrect password. Cannot display the original message.")


Enter your secret message:  Hello this message is secret.
Enter a password for encryption:  1324


✅ Message successfully embedded in stego_image.png

🔁 Re-enter your password to unlock the message.


Re-enter password:  1324



🔓 Original message: Hello this message is secret.
